In [2]:
import abc
import datetime
import timeit
from dataclasses import dataclass
from typing import Literal, Optional

import matplotlib.colors as clr
import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.version
from torch import Tensor

from model.pixelcnn import PixelCNN
from modules.df_attention import DfAttention
from modules.df_conv2d import DfConv2d
from modules.df_elemwise import DfElemwise


ModuleNotFoundError: No module named 'model'

: 

: 

In [ ]:



@torch.no_grad()
def sample(
    depth_first: bool,
    device: str = "cuda",
    dtype: torch.dtype = torch.float32,
    batch_size: int = 16,
    img_size: int = 64,
    in_channels: int = 3,
    hidden_channels: int = 64,
    num_classes: int = 256,
    kernel_size: int = 3,
    seed: Optional[int] = None,
) -> tuple[Tensor, int]:
    if seed == None:
        seed = torch.seed()
    else:
        torch.manual_seed(seed)

    img = (
        torch.randint(
            0,
            num_classes,
            (1, in_channels, img_size, img_size),
            dtype=dtype,
            device=device,
        )
        / num_classes
    ).repeat(batch_size, 1, 1, 1)

    model = PixelCNN(in_channels, hidden_channels, num_classes=num_classes)

    # conv_in = DfConv2d(
    #     in_channels,
    #     hidden_channels,
    #     kernel_size,
    #     device=device,
    #     dtype=dtype,
    #     masked=True,
    # ).eval()
    # act_in = DfElemwise(nn.ReLU())

    # conv_hid = DfConv2d(
    #     hidden_channels,
    #     hidden_channels,
    #     kernel_size,
    #     dilation=1,
    #     device=device,
    #     dtype=dtype,
    #     masked=True,
    # ).eval()
    # act_hid = DfElemwise(nn.ReLU())

    # attn = DfAttention(hidden_channels, device=device, dtype=dtype)

    # conv_out = DfConv2d(
    #     hidden_channels,
    #     num_classes * in_channels,
    #     kernel_size=1,
    #     device=device,
    #     dtype=dtype,
    #     masked=False,
    # ).eval()
    # act_out = DfElemwise(nn.ReLU())

    for y in range(img_size):
        for x in range(img_size):
            pos = (y, x) if depth_first else None
            # o = conv_in(img, pos=pos)
            # o = act_in(o, pos=pos)

            # o = conv_hid(o, pos=pos)
            # o = act_hid(o, pos=pos)

            # o = attn(o, pos=pos)

            # o = conv_out(o, pos=pos)
            # o = act_out(o, pos=pos)
            o = model(img, pos=pos)

            px = (
                o[..., y, x].view(batch_size, in_channels, num_classes).argmax(-1)
                / num_classes
            ) + 1e-15

            img[..., y, x] = px

    plt.imshow(img[0].to(torch.float32).permute(1, 2, 0).cpu(), norm=clr.Normalize())
    return img, seed
